In [ ]:
# ! pip install -U weaviate-client

In [16]:
# import weaviate
# from weaviate.classes.init import Auth 

# # Best practice: store your credentials in environment variables
# weaviate_url = 'https://dbvzpt0resxqjgufx5ing.c0.us-west3.gcp.weaviate.cloud'
# weaviate_api_key = ''

# # Connect to Weaviate Cloud
# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=weaviate_url, 
#     auth_credentials=Auth.api_key(weaviate_api_key),
# )

# print(client.is_ready()) 

True


In [1]:
import weaviate
 
client = weaviate.connect_to_local()

print(client.is_ready())

True


D:\project\weaviate-python-client\weaviate\warnings.py:133: DeprecationWarning: Dep005: You are using weaviate-client version 0.1.dev3116+geac93c3. The latest version is 4.10.2.
            Consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [19]:
# client.collections.delete("OpenVidContext") 


In [26]:
from weaviate.classes.config import Configure, Property, DataType, VectorDistances
 
client.collections.create(
    "OpenVidContext",
    
    vectorizer_config=Configure.Vectorizer.text2vec_contextionary( vectorize_collection_name=False),
 
    properties=[  # Define properties
        Property(name="Video", data_type=DataType.TEXT),
        Property(name="Caption", data_type=DataType.TEXT), 
        
    ],
)

In [2]:
 
import os
import requests

file_path = "OpenVid-1M.csv"

# 检查文件是否存在，如果不存在则下载
if not os.path.exists(file_path):
    print("文件不存在，正在下载...")
    response = requests.get(
        "https://huggingface.co/datasets/nkp37/OpenVid-1M/resolve/main/data/train/OpenVid-1M.csv"
    )
    # 保存文件
    with open(file_path, "wb") as f:
        f.write(response.content)
    print("文件下载完成！")
else:
    print("文件已存在，无需下载。")

# 确认文件状态
file_exists = os.path.exists(file_path)
file_exists
 

文件已存在，无需下载。


True

In [ ]:
import pandas as pd
# Settings for displaying the import progress
counter = 0
interval = 200  # print progress every this many records; should be bigger than the batch_size
collection = client.collections.get("OpenVidContext")

print("JSON streaming, to avoid running out of memory on large files...")
  
 
# 分块读取 CSV 文件
chunk_size = 10000   # 每次读取10000行
for idx, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
    print(f"Processing chunk {idx + 1}...")
    for _, row in chunk.iterrows():
        if idx<1:
            continue
        # 读取每一行的视频信息和标题等
        video_filename = row['video']
        caption = row['caption']
        
 
        # 构建数据
        properties = {
            "Video": video_filename,
            "Caption": caption,
           
        }

        # 向量数据库插入数据
        collection.data.insert(
            properties=properties, 
        )

        counter += 1

        # 每处理200条数据输出一次进度
        if counter % interval == 0:
            print(f"Imported {counter} records...")
            
# 完成后输出总进度
print(f"Finished importing {counter} records.")

JSON streaming, to avoid running out of memory on large files...
Processing chunk 1...
Processing chunk 2...


In [30]:

collection_shards = collection.config.get_shards()
print(collection_shards)

[_ShardStatus(name='Z2QpGRtlElOA', status='READY', vector_queue_size=0)]


In [31]:
for item in collection.iterator():
    print(item.uuid, item.properties)

0001c0da-3ea8-42b0-8a7e-8cf68fc40c0a {'caption': "In the video, two young women are seen in a car, engaging in a casual conversation. The woman on the left is holding up a pink and yellow striped shirt, examining it closely. She appears to be considering whether to purchase it or not. The woman on the right is drinking water from a bottle, seemingly uninterested in the shirt. The car's interior is visible, with the dashboard and steering wheel in the foreground. The women are seated comfortably, suggesting a relaxed and informal atmosphere. The video captures a typical moment of decision-making and everyday life.", 'video': '3QehIbzwOd0_40_0to121.mp4'}
00036ca0-4cd8-4ae7-be6b-cc7de63ff118 {'caption': "The video shows a young man in a black t-shirt with a graphic design on it, standing next to an open car hood. The car is a silver sedan with a red engine cover. The man is looking at the engine, which is visible through the open hood. The car is parked on a concrete surface, and there is

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

